In [14]:
from sympy import *  
init_printing() 
import numpy as np

In [15]:
A = Matrix([[1, 2, 3], 
            [4, 5, 6], 
            [7,8,9]])

A

⎡1  2  3⎤
⎢       ⎥
⎢4  5  6⎥
⎢       ⎥
⎣7  8  9⎦

In [16]:
# To find the transpose:

A.T

⎡1  4  7⎤
⎢       ⎥
⎢2  5  8⎥
⎢       ⎥
⎣3  6  9⎦

In [17]:
# To construct an identity matrix:

display(eye(2))

display(eye(4))

⎡1  0⎤
⎢    ⎥
⎣0  1⎦

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

In [18]:
# To take the determinant:

A.det()

The following two cells might be useful for solving Huckel matrices related problems.

In [19]:
x, alpha, beta = symbols("x alpha beta")

A = Matrix([[x, 1, 0, 0, 1],
       [1, x, 1, 0, 0],
       [0, 1, x, 1, 0],
       [0, 0, 1, x, 1],
       [1, 0, 0, 1, x]]
      )

A

⎡x  1  0  0  1⎤
⎢             ⎥
⎢1  x  1  0  0⎥
⎢             ⎥
⎢0  1  x  1  0⎥
⎢             ⎥
⎢0  0  1  x  1⎥
⎢             ⎥
⎣1  0  0  1  x⎦

In [20]:
A.det()

# --------------------------------------------------------------------------------

## Based on the Spin Adapted Configurations Document 

In [21]:
s = Function('s')
d = Function('d')
phi = Function('phi')
i, j, a, b, a1, b1 = symbols("i, j, a, b, a1, b1")

Looking at a way to immediately transform a problem in dirac notation to a matrix notation... So far, unsuccessful/inefficient.

In [22]:
from sympy.physics.quantum import Bra, Ket, TensorProduct

Bra(phi(i, j))*Ket(phi(a, b) - phi(a1, b1)) 


In [23]:
def dirac_expand_1(bra, bra1, ket, ket1):
    if bra1 == 1:
        return Bra(bra)*Ket(ket) + Bra(bra)*Ket(ket1)
    else:
        return Bra(bra)*Ket(ket) + Bra(bra1)*Ket(ket) + Bra(bra)*Ket(ket1) + Bra(bra1)*Ket(ket1)


display(dirac_expand_1(phi(i, j), 1, phi(a, b), -phi(a1, b1)))


def dirac_to_matrix(var1, var2, var3, var4, var5 = None, var6 = None):
    if var5 == None and var6 == None:
        return Matrix([
            [(var1)*(var3), (var1)*(var4)],
            [(var2)*(var3), (var2)*(var4)]
                                        ])
    else:
        return Matrix([
            [var1*var4, var1*var5, var1*var6],
            [var2*var4, var2*var5, var2*var6],
            [var3*var4, var3*var5, var3*var6]
                                            ])


dirac_to_matrix(phi(i), phi(j), phi(a), phi(b), phi(a1), phi(b1))

⎡φ(b)⋅φ(i)  φ(a₁)⋅φ(i)  φ(b₁)⋅φ(i)⎤
⎢                                 ⎥
⎢φ(b)⋅φ(j)  φ(a₁)⋅φ(j)  φ(b₁)⋅φ(j)⎥
⎢                                 ⎥
⎣φ(a)⋅φ(b)  φ(a)⋅φ(a₁)  φ(a)⋅φ(b₁)⎦

The following A1 and A2 matrices are part of the problem illustrated in the "closed-shell open-shell singlet" problem from the spin adapted configurations document.

If the code can eventually work with the 2-electron system, it should be able to run 3-4 electron systems, as well.

Please also note that as of now, I am unable to "switch" the subscripts in problems, and therefore all of the wave functions of interest are described by functions rather than subscripts (hence the phi(i) notation versus phi_i)

In [24]:
A1 =  Matrix([
    [phi(i)*phi(a), phi(i)*phi(b)],
    [phi(j)*phi(a), phi(j)*phi(b)]
])

A2 =  Matrix([
    [phi(i)*phi(a1), phi(i)*phi(b1)],
    [phi(j)*phi(a1), phi(j)*phi(b1)]
])

display(A1, A2)

⎡φ(a)⋅φ(i)  φ(b)⋅φ(i)⎤
⎢                    ⎥
⎣φ(a)⋅φ(j)  φ(b)⋅φ(j)⎦

⎡φ(a₁)⋅φ(i)  φ(b₁)⋅φ(i)⎤
⎢                      ⎥
⎣φ(a₁)⋅φ(j)  φ(b₁)⋅φ(j)⎦

The following potential code is based off of the methods by Brendan Smith for solving higher-electron problems efficiently. 

A series of matrices are set up, and the columns are either "d" or "s" notation, and the "d" column basically shifts down the matrix, therefore creating multiple matrices. (Not sure if that makes any sense)

This is shown with the example at the bottom of the cell.

In [25]:
n = Symbol("n")

    
def Matrix_transform_1(matrix, dim):
    if dim == 2:
        Matrix_1 = matrix.col(0).replace(phi, d)
        Matrix_2 = matrix.col(1).replace(phi, s)
        
        return Matrix([[Matrix_1, Matrix_2]])
    
    if dim == 3:
        Matrix_1 = matrix.col(0).replace(phi, d)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3]])
    
    else:
        Matrix_1 = matrix.col(0).replace(phi, d)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, s)
        Matrix_4 = matrix.col(3).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3, Matrix_4]])

def Matrix_transform_2(matrix, dim):
    if dim == 2:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, d)
        
        return Matrix([[Matrix_1, Matrix_2]])
    
    if dim == 3:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, d)
        Matrix_3 = matrix.col(2).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3]])
    
    else:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, d)
        Matrix_3 = matrix.col(2).replace(phi, s)
        Matrix_4 = matrix.col(3).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3, Matrix_4]])


def Matrix_transform_3(matrix, dim):
    
    
    if dim == 2:
        return 0
    
    if dim == 3:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, d)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3]])
    
    else:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, d)
        Matrix_4 = matrix.col(3).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3, Matrix_4]])

def Matrix_transform_4(matrix, dim):
    
    if dim == 2 or dim == 3:
        return 0
    
    else:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, s)
        Matrix_4 = matrix.col(3).replace(phi, d)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3, Matrix_4]])


def matrix_transform(matrix, dim):
    if dim == 2:
        return TensorProduct(Matrix_transform_1(matrix, dim) + Matrix_transform_2(matrix, dim), eye(dim))
    if dim == 3:
        return display(TensorProduct(Matrix_transform_1(matrix, dim))) , display(TensorProduct(Matrix_transform_2(matrix, dim))) , display(TensorProduct(Matrix_transform_3(matrix, dim), eye(dim)))
    if dim == 4:
        return display(TensorProduct(Matrix_transform_1(matrix, dim), eye(dim))) , display(TensorProduct(Matrix_transform_2(matrix, dim), eye(dim))) , display(TensorProduct(Matrix_transform_3(matrix, dim), eye(dim))) , display(TensorProduct(Matrix_transform_4(matrix, dim), eye(dim)))


# R = simplify(Matrix_transform_1(B) + Matrix_transform_2(B))

a, b, c, e, i, j, k, l = symbols("a, b, c, e, i, j, k, l")
psi = Function('psi')

M = Matrix([[phi(a), phi(b), phi(c), phi(e)],
       [phi(a), phi(b), phi(c), phi(e)],
       [phi(i), phi(j), phi(k), phi(l)],
           [phi(a), phi(l), phi(k), phi(e)]
           ])

matrix_transform(M, 4)

⎡d(a)  s(b)  s(c)  s(e)⎤
⎢                      ⎥
⎢d(a)  s(b)  s(c)  s(e)⎥
⎢                      ⎥
⎢d(i)  s(j)  s(k)  s(l)⎥
⎢                      ⎥
⎣d(a)  s(l)  s(k)  s(e)⎦

⎡s(a)  d(b)  s(c)  s(e)⎤
⎢                      ⎥
⎢s(a)  d(b)  s(c)  s(e)⎥
⎢                      ⎥
⎢s(i)  d(j)  s(k)  s(l)⎥
⎢                      ⎥
⎣s(a)  d(l)  s(k)  s(e)⎦

⎡s(a)  s(b)  d(c)  s(e)⎤
⎢                      ⎥
⎢s(a)  s(b)  d(c)  s(e)⎥
⎢                      ⎥
⎢s(i)  s(j)  d(k)  s(l)⎥
⎢                      ⎥
⎣s(a)  s(l)  d(k)  s(e)⎦

⎡s(a)  s(b)  s(c)  d(e)⎤
⎢                      ⎥
⎢s(a)  s(b)  s(c)  d(e)⎥
⎢                      ⎥
⎢s(i)  s(j)  s(k)  d(l)⎥
⎢                      ⎥
⎣s(a)  s(l)  s(k)  d(e)⎦

(None, None, None, None)

Now we can try the same method with the A1 and A2 matrices we defined previously.

The "determinat()" function was defined for easy simplification. 

Please note that lines 14 and 16 are equivalent, but in line 16 the "subscripts" were simplified by hand for simplicity. (Again, keep in mind as of now I cannot work with subscripts and therefore everything is defined by functions)

Please also note that the 1/sqrt(2) is not included in this example (as of now, it simplifies incorrectly, and I am unsure why that happens)

In [26]:
T1 = matrix_transform(A1, 2)
T2 = matrix_transform(A2, 2)

display(factor(T1), factor(T2))

expand(det(factor(simplify(T1))) + det(factor(simplify(T2))))


def determinant(matrix):
    return expand(det(factor(simplify(matrix))))

q = determinant(T1) + determinant(T2)

display(q)

d(a, i)*s(b, j) - d(a, j)*s(b, i) + d(a1, i)*s(b1, j) - d(a1, j)*s(b1, i) - d(b, i)*s(a, j) + d(b, j)*s(a, i) - d(b1, i)*s(a1, j) + d(b1, j)*s(a1, i)


⎡d(a)⋅d(i) + s(a)⋅s(i)  d(b)⋅d(i) + s(b)⋅s(i)⎤
⎢                                            ⎥
⎣d(a)⋅d(j) + s(a)⋅s(j)  d(b)⋅d(j) + s(b)⋅s(j)⎦

⎡d(a₁)⋅d(i) + s(a₁)⋅s(i)  d(b₁)⋅d(i) + s(b₁)⋅s(i)⎤
⎢                                                ⎥
⎣d(a₁)⋅d(j) + s(a₁)⋅s(j)  d(b₁)⋅d(j) + s(b₁)⋅s(j)⎦

Note: need to break it into two matrices to compute properly.

In the below cell, I try to add in the 1/sqrt(2) factor (please note that this would not be correct fully, since some terms have 1/sqrt(2) and others have -1/sqrt(2))

I also tried to replace the terms with those listed in the document for simplification, which leads to some interesting results.

In [29]:
qr = q.replace(i, alpha).replace(j, beta).replace(a, 2*alpha).replace(b, beta).replace(a1, 2*beta).replace(b1, alpha)

simplify((1/sqrt(2))*qr)

I then try splitting up the determinants back into the T1 and T2 determinants which were mentioned previously, hoping that would work. They were then added together as per usual (it might be a good idea to include the 1/sqrt(2) and -1/sqrt(2) coefficients here?)

In [28]:
S = (T1.replace(i, alpha).replace(j, beta).replace(a, 2*alpha).replace(b, beta).replace(a1, 2*beta).replace(b1, alpha))
m = (T2.replace(i, alpha).replace(j, beta).replace(a, 2*alpha).replace(b, beta).replace(a1, 2*beta).replace(b1, alpha))

expand(determinant(S) + determinant(m))

The below code is quite muddy. It is based off of some of the code from Brendan Smith's presentation.

Trying three electron systems...

In [30]:
M = Matrix([
    [phi(alpha)*phi(alpha), phi(alpha)*phi(beta), phi(alpha)*phi(3*alpha)],
    [phi(2*beta)*phi(alpha), phi(2*beta)*phi(beta), phi(2*beta)*phi(3*alpha)],
    [phi(3*alpha)*phi(alpha), phi(3*alpha)*phi(beta), phi(3*alpha)*phi(3*alpha)]
])

display(M)

N1 = Matrix_transform_1(M, 3).replace(s(alpha)*s(beta), 0).replace(s(3*alpha)*s(beta), 0).replace(d(alpha)*d(2*beta), 0).replace(s(3*alpha)*s(2*beta), 0)# , \
N2 = Matrix_transform_2(M, 3).replace(s(alpha)*s(2*beta), 0).replace(d(alpha)*d(beta), 0).replace(s(3*alpha)*s(2*beta), 0).replace(d(3*alpha)*d(beta), 0) #, \
N3 = Matrix_transform_3(M, 3).replace(s(alpha)*s(2*beta), 0).replace(s(alpha)*s(beta), 0).replace(s(3*alpha)*s(beta), 0).replace(d(3*alpha)*d(2*beta), 0)


RF = TensorProduct(N1, eye(3)) + TensorProduct(N2, eye(3)) + TensorProduct(N3, eye(3))

expand(det(RF))

⎡    2                                  ⎤
⎢   φ (α)      φ(α)⋅φ(β)    φ(α)⋅φ(3⋅α) ⎥
⎢                                       ⎥
⎢φ(α)⋅φ(2⋅β)  φ(β)⋅φ(2⋅β)  φ(3⋅α)⋅φ(2⋅β)⎥
⎢                                       ⎥
⎢                              2        ⎥
⎣φ(α)⋅φ(3⋅α)  φ(3⋅α)⋅φ(β)     φ (3⋅α)   ⎦

In [31]:
omega = Symbol("Omega")

M = Matrix([
    [phi(a)*phi(a), phi(a)*phi(b), phi(a)*phi(omega)],
    [phi(2*b)*phi(a), phi(2*b)*phi(b), phi(2*b)*phi(omega)],
    [phi(omega)*phi(a), phi(omega)*phi(b), phi(omega)*phi(omega)]
])

M

Q = Matrix_transform_1(M, 3)
X = Matrix_transform_2(M, 3)
Y = Matrix_transform_3(M, 3)

In [32]:
RF = ((TensorProduct(Q, eye(3)) + TensorProduct(X, eye(3)) + TensorProduct(Y, eye(3))))

RF

⎡       2         2                                                           
⎢      d (a) + 2⋅s (a)          d(a)⋅d(b) + 2⋅s(a)⋅s(b)      d(Ω)⋅d(a) + 2⋅s(Ω
⎢                                                                             
⎢d(a)⋅d(2⋅b) + 2⋅s(a)⋅s(2⋅b)  d(b)⋅d(2⋅b) + 2⋅s(b)⋅s(2⋅b)  d(Ω)⋅d(2⋅b) + 2⋅s(Ω
⎢                                                                             
⎢                                                                 2         2 
⎣  d(Ω)⋅d(a) + 2⋅s(Ω)⋅s(a)      d(Ω)⋅d(b) + 2⋅s(Ω)⋅s(b)          d (Ω) + 2⋅s (

        ⎤
)⋅s(a)  ⎥
        ⎥
)⋅s(2⋅b)⎥
        ⎥
        ⎥
Ω)      ⎦

Please note as well, that the below is for general electrons in symbolic notation, instead of known electron states as shown in Brendan Smith's work.

In [33]:
M = Matrix([
    [phi(a)*phi(i), phi(a)*phi(j), phi(a)*phi(k)],
    [phi(b)*phi(i), phi(b)*phi(j), phi(b)*phi(k)],
    [phi(c)*phi(i), phi(c)*phi(j), phi(c)*phi(k)]
])

display(M)

a, b, c, i, j, k = symbols("a, b, c, i, j, k")

⎡φ(a)⋅φ(i)  φ(a)⋅φ(j)  φ(a)⋅φ(k)⎤
⎢                               ⎥
⎢φ(b)⋅φ(i)  φ(b)⋅φ(j)  φ(b)⋅φ(k)⎥
⎢                               ⎥
⎣φ(c)⋅φ(i)  φ(c)⋅φ(j)  φ(c)⋅φ(k)⎦

The below cells for just the matrix_transform() function do not seem to be working (the matrix_transform function separates the matrix into its columns to change each column individually, then rejoins the columns. The last matrix is correct, but the first two are not).

In [38]:
matrix_transform(M, 3)

⎡⎡d(a)⋅d(i)⎤  ⎡s(a)⋅s(j)⎤  ⎡s(a)⋅s(k)⎤⎤
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎢⎢d(b)⋅d(i)⎥  ⎢s(b)⋅s(j)⎥  ⎢s(b)⋅s(k)⎥⎥
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎣⎣d(c)⋅d(i)⎦  ⎣s(c)⋅s(j)⎦  ⎣s(c)⋅s(k)⎦⎦

⎡⎡s(a)⋅s(i)⎤  ⎡d(a)⋅d(j)⎤  ⎡s(a)⋅s(k)⎤⎤
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎢⎢s(b)⋅s(i)⎥  ⎢d(b)⋅d(j)⎥  ⎢s(b)⋅s(k)⎥⎥
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎣⎣s(c)⋅s(i)⎦  ⎣d(c)⋅d(j)⎦  ⎣s(c)⋅s(k)⎦⎦

⎡s(a)⋅s(i)  s(a)⋅s(j)  d(a)⋅d(k)⎤
⎢                               ⎥
⎢s(b)⋅s(i)  s(b)⋅s(j)  d(b)⋅d(k)⎥
⎢                               ⎥
⎣s(c)⋅s(i)  s(c)⋅s(j)  d(c)⋅d(k)⎦

(None, None, None)

Change definition of matrix transform. Have to do each separately. (?) What if you don't have actual values for the electrons?

In [39]:
det(matrix_transform(M, 3))

⎡⎡d(a)⋅d(i)⎤  ⎡s(a)⋅s(j)⎤  ⎡s(a)⋅s(k)⎤⎤
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎢⎢d(b)⋅d(i)⎥  ⎢s(b)⋅s(j)⎥  ⎢s(b)⋅s(k)⎥⎥
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎣⎣d(c)⋅d(i)⎦  ⎣s(c)⋅s(j)⎦  ⎣s(c)⋅s(k)⎦⎦

⎡⎡s(a)⋅s(i)⎤  ⎡d(a)⋅d(j)⎤  ⎡s(a)⋅s(k)⎤⎤
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎢⎢s(b)⋅s(i)⎥  ⎢d(b)⋅d(j)⎥  ⎢s(b)⋅s(k)⎥⎥
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎣⎣s(c)⋅s(i)⎦  ⎣d(c)⋅d(j)⎦  ⎣s(c)⋅s(k)⎦⎦

⎡s(a)⋅s(i)  s(a)⋅s(j)  d(a)⋅d(k)⎤
⎢                               ⎥
⎢s(b)⋅s(i)  s(b)⋅s(j)  d(b)⋅d(k)⎥
⎢                               ⎥
⎣s(c)⋅s(i)  s(c)⋅s(j)  d(c)⋅d(k)⎦

TypeError: Input to Determinant, (None, None, None), not a matrix